In [4]:
import slicerutil as su
import slicer
import os 
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from slicerutil import PresetColormaps, PetColormaps

In [5]:
data_path = r"C:\Users\milantom\OneDrive - LHSC & St. Joseph's\Documents\Eso_deep_learning\ESO_DL_DS"
clinical_csv = r"C:\Users\milantom\OneDrive - LHSC & St. Joseph's\Documents\Eso_deep_learning\csv_files\Clinical_Data_Eso.csv"

new_npy_save_path = r"C:\Users\milantom\OneDrive - LHSC & St. Joseph's\Documents\Eso_deep_learning\ESO_DL_DS_resampled"

In [6]:
patient_image_paths_dict = {}

for year_folder in os.listdir(data_path):
    year_path = os.path.join(data_path, year_folder)
    for modality_folder in os.listdir(year_path):
        modality_path = os.path.join(year_path, modality_folder)

        patient_id = f"LRCP_{modality_folder.split('.')[1].split('_')[0]}"
        if patient_id not in patient_image_paths_dict.keys():
            patient_image_paths_dict[patient_id] = {}

        if '_CT_' in modality_folder:
            patient_image_paths_dict[patient_id]['CT'] = modality_path
        elif '_PT_' in modality_folder:
            patient_image_paths_dict[patient_id]['PET'] = modality_path 
        else:
            print(f"Unknown modality: {modality_folder}")
       

In [7]:
if not os.path.exists(new_npy_save_path):
    os.makedirs(new_npy_save_path)

CT_npy_locations = []
PET_SUVbw_npy_locations = []
PET_SUVlbm_npy_locations = []
PET_SUVbsa_npy_locations = []

count = 0
for patient in tqdm(patient_image_paths_dict.keys()):
    slicer.mrmlScene.Clear(0)
    print(f"Processing patient: {patient}")
    count += 1
    if count <= 116:
        continue

    # Create a new directory for the patient
    patient_folder = os.path.join(new_npy_save_path, patient)
    if not os.path.exists(patient_folder):
        os.makedirs(patient_folder)
    
    # Load the clinical data
    CT_node_ids = su.load_DICOM(patient_image_paths_dict[patient]['CT'])
    PET_node_ids = su.load_DICOM(patient_image_paths_dict[patient]['PET'])

    CT_volume = su.ImageVolume(slicer.util.getNode(CT_node_ids[0]))
    CT_Numpy = CT_volume.NumPyArray

    PET_images_exist = {'bw': False, 'lbm': False, 'bsa': False}
    PET_image_numpy = {'bw': None, 'lbm': None, 'bsa': None}

    for node_id in PET_node_ids:
        PET_node_name = slicer.util.getNode(node_id).GetName()

        if '(SUVbw)' in PET_node_name:
            PET_volume_bw = su.ImageVolume(slicer.util.getNode(node_id))
            PET_images_exist['bw'] = PET_volume_bw
        
        elif '(SUVlbm)' in PET_node_name:
            PET_volume_lbm = su.ImageVolume(slicer.util.getNode(node_id))
            PET_images_exist['lbm'] = PET_volume_lbm

        elif '(SUVbsa)' in PET_node_name:
            PET_volume_bsa = su.ImageVolume(slicer.util.getNode(node_id))
            PET_images_exist['bsa'] = PET_volume_bsa

        if all(PET_images_exist.values()):
            break
            
    for key, value in PET_images_exist.items():
        if not value:
            print(f"PET image with {key} does not exist.")
        else:
            temp_resampled = value.resample_scalar_volume_BRAINS(CT_volume.volumeNode, f"SUV{key}_resampled_BSpline", interpolatorType='BSpline')
            PET_image_numpy[key] = su.ImageVolume(temp_resampled).NumPyArray

    
    # Save the resampled images
    np.save(os.path.join(patient_folder, f"{patient}_CT.npy"), CT_Numpy)
    np.save(os.path.join(patient_folder, f"{patient}_PET_SUVbw.npy"), PET_image_numpy['bw'])
    np.save(os.path.join(patient_folder, f"{patient}_PET_SUVlbm.npy"), PET_image_numpy['lbm'])
    np.save(os.path.join(patient_folder, f"{patient}_PET_SUVbsa.npy"), PET_image_numpy['bsa'])

    CT_npy_locations.append(os.path.join(patient_folder, f"{patient}_CT.npy"))
    PET_SUVbw_npy_locations.append(os.path.join(patient_folder, f"{patient}_PET_SUVbw.npy"))
    PET_SUVlbm_npy_locations.append(os.path.join(patient_folder, f"{patient}_PET_SUVlbm.npy"))
    PET_SUVbsa_npy_locations.append(os.path.join(patient_folder, f"{patient}_PET_SUVbsa.npy"))

    del CT_volume
    del PET_volume_bw
    del PET_volume_lbm
    del PET_volume_bsa
    del PET_image_numpy['bw']
    del PET_image_numpy['lbm']
    del PET_image_numpy['bsa']

    del PET_images_exist['bw']
    del PET_images_exist['lbm']
    del PET_images_exist['bsa']

    del PET_node_ids
    del CT_node_ids
    del CT_Numpy
    del PET_image_numpy
    del PET_images_exist
    del PET_node_name

# Create a DataFrame to store the file paths
df = pd.DataFrame({
    'Patient_ID': list(patient_image_paths_dict.keys()),
    'CT': CT_npy_locations,
    'PET_SUVbw': PET_SUVbw_npy_locations,
    'PET_SUVlbm': PET_SUVlbm_npy_locations,
    'PET_SUVbsa': PET_SUVbsa_npy_locations
})

# Save the DataFrame to a CSV file
pd.save_csv(df, os.path.join(new_npy_save_path, "npy_save_paths_2.csv"), index=False)


  0%|          | 0/164 [00:00<?, ?it/s]

Processing patient: LRCP_ESO165


  1%|          | 1/164 [00:00<00:17,  9.20it/s]

Processing patient: LRCP_ESO162


  1%|1         | 2/164 [00:00<00:17,  9.30it/s]

...

Processing patient: LRCP_ESO130
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031


 99%|#########9| 163/164 [3:36:54<04:45, 285.27s/it]

Processing patient: LRCP_ESO131
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Radiopharmaceutical Code C-B1031
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Referenced PET series
Found Radiopharmaceutical Code C-B1031


100%|##########| 164/164 [3:41:16<00:00, 80.96s/it] 


<class 'ValueError'>: All arrays must be of the same length